# Scraping score and odds data

## Soccer score

In [17]:
from bs4 import BeautifulSoup
from urllib import request
import requests
import json
import pandas as pd
import datetime
import time

In [2]:
# 此处插入日期生成器 2013-2-1 到 2018-5-30
numdays = 1900
startdate = datetime.date(2013,2,1)
date_list = [(startdate + datetime.timedelta(days=x)).isoformat() for x in range(0, numdays)]

In [15]:
## soccer request
def add_newevents(date_range,events_dic):
    for i in date_range:
        url = 'https://io.oddsshark.com/scores/mls/'+ date_list[i]
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36',
        'Authorization': 'io.oddsshark.com',
        'Path': '/scores/mls/'+ date_list[i],
        'Referer': 'https://www.oddsshark.com/soccer/mls/scores',
        'Connection': 'keep-alive'
        }
        req = request.Request(url, headers=headers)
        page = request.urlopen(req).read().decode('utf-8')

        jdata = json.loads(page)
        for data in jdata:

            event_id = data['event_id']
            event_date = data['event_date']

            if data.get('home_name'):
                home_display_name = data['home_name']     
            elif data.get('home_display_name'):
                home_display_name = data['home_display_name']
            else:
                continue
            home_score = data['home_score']

            if data.get('away_name'):
                away_display_name = data['away_name']
            elif data.get('away_display_name'):  
                away_display_name = data['away_display_name']
            else:
                continue
            away_score = data['away_score']

            events_dic[event_id] = [event_date,home_display_name,home_score,away_display_name,away_score]
    return events_dic

In [ ]:
#  加个代理 proxy

In [19]:
events_dic = {}
for r in range(2):
    events_dic = add_newevents(range(r*100,(r+1)*100),events_dic)
    time.sleep(5)
    print(len(events_dic))

# columns=['event_date', 'home_name', 'home_score', 'away_name','away_score']
df = pd.DataFrame.from_dict(events_dic, orient='index').rename(index=str, 
                                                               columns={0: "event_date",
                                                                        1: "home_name",
                                                                        2: "home_score",
                                                                        3: "away_name",
                                                                        4: "away_score"})
df

98
229


,event_date,home_name,home_score,away_name,away_score
557831,2013-08-17 19:30,New England,2,Chicago,0
558093,2013-06-01 22:30,CD Chivas USA,0,Seattle,2
631936,2013-06-15 17:00,Portland,1,Dallas,0
557816,2013-05-05 17:00,Kansas City,4,CD Chivas USA,0
558019,2013-04-27 19:30,New England,2,Philadelphia,0
558037,2013-03-30 18:00,Colorado,2,Portland,2
558082,2013-03-30 21:00,Real Salt Lake,2,Seattle,1
557856,2013-04-14 12:00,Montreal,1,Columbus,1
557989,2013-06-29 22:30,CD Chivas USA,1,New England,1
558046,2013-04-13 16:00,Vancouver,1,Real Salt Lake,1


In [21]:
df.to_csv("../datasets/soccer/mls_results.csv")

## odds

In [50]:
url = 'https://www.oddsshark.com/soccer/mls/odds/line-history/953912'
req = request.Request(url)
page1 = request.urlopen(req).read()
page1 = page1.decode('utf-8')

In [81]:
soup = BeautifulSoup(page1)
odata = soup.find_all('table',{'class':'base-table'})
# odata = json.loads(page1)

odict = {}
for data in odata:
#     soup = BeautifulSoup(data)
    booker = data.find('a').get_text()
    content = data.find_all('td')
    oodict = {}
    for i in range(int(len(content)/5)):
#         此处加一个换时间的
        date = content[i*5].get_text()
        home = content[i*5+1].find('span').get_text()
        away = content[i*5+2].find('span').get_text()
        draw = content[i*5+3].find('span').get_text()
        if home!="" and away!="" and draw!="":
            oodict[date] = {'home':home,'away':away,'draw':draw}
    odict[booker] = oodict

    print(booker)
    print(oodict)
    print()
    
    

C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\surface\Anaconda2\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Bet365
{'05/29/18 9:22:40 AM': {'home': '+160', 'away': '+160', 'draw': '+250'}, '05/28/18 3:15:24 AM': {'home': '+160', 'away': '+160', 'draw': '+240'}, '05/30/18 6:39:10 PM': {'home': '+175', 'away': '+162', 'draw': '+250'}, '05/30/18 6:39:09 PM': {'home': '+175', 'away': '+140', 'draw': '+250'}, '05/29/18 12:44:15 PM': {'home': '+160', 'away': '+160', 'draw': '+250'}, '05/30/18 5:25:57 PM': {'home': '+175', 'away': '+140', 'draw': '+250'}, '05/29/18 7:20:51 PM': {'home': '+162', 'away': '+150', 'draw': '+250'}, '05/28/18 4:29:00 PM': {'home': '+188', 'away': '+138', 'draw': '+240'}, '05/30/18 6:52:51 PM': {'home': '+150', 'away': '+162', 'draw': '+250'}, '05/30/18 3:35:31 PM': {'home': '+162', 'away': '+150', 'draw': '+250'}, '05/29/18 8:34:20 AM': {'home': '+180', 'away': '+160', 'draw': '+250'}, '05/30/18 4:16:57 PM': {'home': '+162', 'away': '+150', 'draw': '+250'}, '05/30/18 3:30:13 PM': {'home': '+162', 'away': '+150', 'draw': '+250'}, '05/28/18 4:57:54 AM': {'home': '+175', 'a